# Homework 4


In [130]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from matplotlib import pyplot as plt
import os.path
from os.path import exists
import os

from sklearn.decomposition import PCA

import time
import numpy as np
import h5py
import urllib.request

device="cpu"
if torch.cuda.is_available():
    torch.backends.cudnn.deterministic = True
    device="cuda:0"
print(device)    

cpu


## Problem 1 (40 points): Revisiting our CNN from lecture

Let's revisit the CNN notebook that we worked through in class, and modify the code there to do something slightly different.

First, let's import our data:

In [131]:
output = 'ising_data.npz'

if not exists(output):
    import gdown
    url = "https://drive.google.com/u/0/uc?export=download&confirm=HgGH&id=1Ihxt1hb3Kyv0IrjHlsYb9x9QY7l7n2Sl"
    gdown.download(url, output, quiet=True)

f = np.load(output, allow_pickle=True)
n_train = 20000
n_test=len(f["C"])-n_train

x_train, x_test = f["C"][:n_train], f["C"][n_train:]
y_train, y_test = f["T"][:n_train], f["T"][n_train:]

x_train_tensor=torch.tensor(x_train,dtype=torch.float).unsqueeze(1)
y_train_tensor=torch.tensor(y_train,dtype=torch.float).unsqueeze(1)

x_test_tensor=torch.tensor(x_test,dtype=torch.float).unsqueeze(1)
y_test_tensor=torch.tensor(y_test,dtype=torch.float).unsqueeze(1)

xy_train = torch.utils.data.TensorDataset(x_train_tensor.float(),y_train_tensor)

torch.manual_seed(123)

In the CNN lecture notebook, we used a CNN to predict whether the image had crossed the phase transition boundary or not.  What I'd like to do is a followup that I may have mentioned in lecture: let's use the CNN to predict the temperature that corresponds to the image, instead of a binary "yes/no" decision.  Some things to consider:

* What should the loss function be?

**Now your job**: setup and train a CNN that predicts the temperature associated with each image.  Here are some requirements, which may be different from what was done in the CNN notebook in class:

* Have the first convolutional layer use 16 filters
* Have the second convolutional layer use 32 filters
* Have the fully-connected part at the end include two layers with 32 nodes
* Use the `Adam` optimizer with a learning rate of 0.01
* Run for 10 epochs


As part of this problem, you should produce:
* The normal loss plot showing the training and test losses vs the number of epochs
* Once the network is trained, pass the test data through the network again, and get the predicted values.  Compare the predicted values with the true values; plot the difference and find the bias and variance of the network.

Some hints:
* You don't necessarily need to run the code for the full 16+32 convolutional filter network every time, the training takes a little while!  Feel free to debug with smaller numbers of filters.
* However, the setup is a bit delicate, so even small changes in the hyperparameters specified above may cause the training to not converge.

In [132]:
# code for creating the neural net

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # two convolutional layers
        self.conv1 = torch.nn.Conv2d( 1, 16, kernel_size=3, padding=1)
        self.conv2 = torch.nn.Conv2d(16, 32, kernel_size=3, padding=1)

        # fully connected layer
        self.fc1 = torch.nn.Linear(32, 32)
        self.fc2 = torch.nn.Linear(32, 1)

    # x represents our data
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)

        # if we wanted to add another layer before pooling, we could do that here.

        # Run max pooling over x
        x = F.max_pool2d(x, 2)
        
        # we could also do some dropout here if we wanted

        x = self.conv2(x)
        x = F.relu(x)

        # perhaps we want another layer here?

        # the (1,1) makes this into global average pooling
        x = F.adaptive_avg_pool2d(x,(1,1))
    
        # Flatten x with start_dim=1, i.e. only keep one number per channel per event.
        # This one line kept me up into the wee hours the other night!
        x=torch.flatten(x,1)
      
        # Pass data through the fully connected layer.
        x = self.fc1(x)
        x = self.fc2(x)

        return x

In [133]:
# create the net, define the loss fuction and optimizer
net = Net()
print(net)
print(np.sum([np.prod(theta.shape) for theta in net.parameters()]))

loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

Net(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=32, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=1, bias=True)
)
5889


In [ ]:
# train the net
losses = []
losses_test = []

loader = torch.utils.data.DataLoader(xy_train, batch_size=int(len(x_train)/5), shuffle=True)

for epoch in range(10):
    start_time = time.time()
    for x_batch, y_batch in loader:
        y_pred = net(x_batch)
        optimizer.zero_grad()
        loss = loss_fn(y_pred, y_batch)
        loss.backward()
        optimizer.step()
    losses.append(loss.data)
    net.eval() # configure the model for evaluation (testing)
    y_pred = net(x_test_tensor)
    test_loss=loss_fn(y_pred, y_test_tensor)
    losses_test.append(test_loss.data)
    end_time=time.time()
    elapsed_time = end_time - start_time
    print("Completed epoch %2d in %4.1fs, Train loss=%4.1e, Test loss=%4.1e" % (epoch, elapsed_time, loss.data, test_loss.data))

In [ ]:
# plot the losses'
plt.plot(losses, '.', label="Train")
plt.plot(losses_test, '.', label="Test")
plt.legend()
plt.xlabel('Training Epoch')
plt.ylabel('MSE Loss')
plt.yscale('log')

In [ ]:
# evaluate and plot the losses for individual training dataset events
y_pred = np.array(net(x_test_tensor).data.flatten())

T_train, T_test = f["T"][:n_train], f["T"][n_train:]
diff = T_test - y_pred

plt.scatter(T_test, y_pred)
plt.xlabel('True Temperatures')
plt.ylabel('PRedicted Temperatures')
plt.show()

plt.hist(diff,bins=100,range=[-3,3])
plt.show()

print("Mean = %f" % np.mean(diff))
print("Variance = %f" % np.var(diff))
print("Std Dev = %f" % np.var(diff)**0.5)

## Problem 2 (20 points): Principal Components Analysis

Let's perform a PCA test on some input data that we'll use for Problem #3.

First let's get some training and test datasets that we can use...  we'll start with one of the files that we used from HW3:

In [123]:
def clean_sample(sample):
    events=[]
    for i in sample:
        if i[0]>2:
            events.append(i)
    return events

def get_sample_from_URL(samplename,branches):
    
    if not os.path.isfile(samplename):
        urllib.request.urlretrieve("http://mhance.scipp.ucsc.edu/%s" % samplename, samplename)

    alldata=None
    with h5py.File(samplename,'r') as hdf5file:
        data=hdf5file[list(hdf5file.keys())[0]]["lowleveltree"]
        num_backgr_events=len(data["numjet"])
        alldata = data[branches]    

    Alldata=[[float(i) for i in j] for j in alldata]
    Alldata_clean=clean_sample(Alldata)
    return Alldata_clean

In [124]:
# now scale based on the training data:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

branches=("numjet","numlepton","numbtagjet",
          "met","metphi",
          "jet1pT", "jet1eta", "jet1phi","jet1b",
          "jet2pT", "jet2eta", "jet2phi","jet2b",
          "jet3pT", "jet3eta", "jet3phi","jet3b")

traindata=get_sample_from_URL('lowlevelAna_ttbar.hf5',branches)
x_train = sc.fit_transform(traindata)
train=torch.tensor(x_train,dtype=torch.float)

testdata=get_sample_from_URL('lowlevelAna_test.hf5',branches)
x_test = sc.transform(testdata)
test=torch.tensor(x_test,dtype=torch.float)

Using the `scikit-learn` PCA tool, analyze the `ttbar` dataset (`x_train`) retrieved above.

* Find the number of components needed to explain 50% of the variance.
* List the magnitudes of that number of leading eigenvalues, as a fraction of the total variance they capture

In [125]:
# your code here

pca = PCA().fit(x_train)
print(np.cumsum(pca.explained_variance_ratio_))
# From the output array it can be seen that 5 components (0th component counted as 1st) are needed to explain >= 50% of the variance

eigenval = pca.explained_variance_
print(eigenval)
var = np.sum(eigenval)
frac = eigenval/var
print(frac) # same as explained_variance_ratio as required.

# first 5 componets are 
for i in range(5):
    print(f'eigenvalue/variance for leading eigenvalue {i} = {frac[i]}')

[0.15115807 0.2541584  0.3479097  0.43165682 0.51334968 0.57964796
 0.64179913 0.70299069 0.76102156 0.8083036  0.85089644 0.88876157
 0.92450541 0.95757566 0.97989425 0.99199516 1.        ]
[2.56995718 1.75118951 1.59393956 1.42385066 1.3889244  1.12718924
 1.0566808  1.04036593 0.98662837 0.80387912 0.72415433 0.64377484
 0.60770917 0.56225322 0.37945587 0.20573718 0.13609652]
[0.15115807 0.10300033 0.0937513  0.08374712 0.08169285 0.06629828
 0.06215116 0.06119157 0.05803087 0.04728204 0.04259284 0.03786513
 0.03574384 0.03307024 0.02231859 0.01210092 0.00800484]
eigenvalue/variance for leading eigenvalue 0 = 0.15115807186715166
eigenvalue/variance for leading eigenvalue 1 = 0.10300032709372527
eigenvalue/variance for leading eigenvalue 2 = 0.0937513018331755
eigenvalue/variance for leading eigenvalue 3 = 0.08374712329075978
eigenvalue/variance for leading eigenvalue 4 = 0.08169285322720388


## Problem 3 (40 points): AutoEncoders for Anomaly Detection

We discussed the use of AutoEncoders for de-noising, let's also look at AutoEncoders in the context of anomaly detection.  The goal will be to use an AutoEncoder to find "anomalies" in a data sample without knowing what those anomalies are.  We'll do this by training the network on an un-labeled data sample (i.e. unsupervised learning) and then using the trained network to evaluate a test dataset that has anomalies included.  The anomalies should show up as events with large loss values.

For this problem, we'll use the `ttbar` dataset from problem 2 for training, and the `testdata` dataset from problem 2 for testing.

**Your job**: Define our AutoEncoder, following the scheme we used in the AutoEncoder notebook in class, with the following changes:

* The number of input variables should be the number of variables needed to explain 50% of the variance in the `ttbar` sample (see solutions to Problem 2, let's call this number N_50).
* Use the `LeakyReLU` activation function, with args = `(0.2,inplace-True)`
* The encoder should have three layers:  (1) the input layer, with N nodes; (2) a hidden layer with 2\*N nodes; (3) a hidden layer with 10 nodes; (4) the latent-space layer, with N_50 nodes.
* Make sure to not define an activation on the last layer!



In [126]:
# your code here

class AutoEncoder(nn.Module):
    def __init__(self,n):
        super().__init__()
        
        self.encoder = torch.nn.Sequential(
                torch.nn.Linear(n, 2*n),
                torch.nn.LeakyReLU(0.2, inplace=True),
                torch.nn.Linear(2*n, 10),
                torch.nn.LeakyReLU(0.2, inplace=True),
                torch.nn.Linear(10, 5),
                torch.nn.LeakyReLU(0.2, inplace=True),
                torch.nn.Linear(5, 5), # latent space layer
                )
            
        self.decoder = nn.Linear(5, n)
        self.data_rho=None

    def forward(self, x):
        x = self.encoder(x)
        self.data_rho = x.mean(0)
        x = self.decoder(x)
        x = torch.sigmoid(x)
        return x  

Now train the autoencoder on the training data.  Run for 1500 epochs, using the `Adam`optimizer with a learning rate of 0.01.  Afterwards:
* Make a plot of the loss function vs epochs for the training data
* Make a plot of the loss *values* for all events in the training dataset.  This means passing single events into the loss function, instead of entire tensors!
* Now put the net into evaluation mode, and pass the test dataset through the network.  Make the same plot of loss values for all events in the test dataset.  
* Make a single plot that includes both histograms: the training set, and the testing set.

The difference between the training data and the test data can be subtle unless you overlay them, and (hopefully) see that there are more events in the large-loss tail of the test distribution than the training distribution!

In [127]:
# training loop
pca = PCA(5)
train = torch.tensor(pca.fit_transform(x_train),dtype=torch.float)


print(x_train.shape)

net = AutoEncoder(5)
print(net)

loss_fn = torch.nn.MSELoss()
print(len(x_train))
optimizer = torch.optim.Adam(net.parameters(), lr=0.01)

for epoch in range(1500):
    start_time = time.time()
    pred = net(x_train)
    optimizer.zero_grad()
    loss = loss_fn(pred, x_train)
    loss.backward()
    optimizer.step()
    losses.append(loss.data)
    
    net.eval() # configure the model for evaluation (testing)
    pred_test = net(x_test)
    test_loss=loss_fn(pred_test, x_test)
    losses_test.append(test_loss.data)
    end_time=time.time()
    elapsed_time = end_time - start_time
    print("Completed epoch %2d in %4.1fs, Train loss=%4.1e, Test loss=%4.1e" % (epoch, elapsed_time, loss.data, test_loss.data))

(9520, 17)
AutoEncoder(
  (encoder): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): LeakyReLU(negative_slope=0.2, inplace=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): LeakyReLU(negative_slope=0.2, inplace=True)
    (4): Linear(in_features=10, out_features=5, bias=True)
    (5): LeakyReLU(negative_slope=0.2, inplace=True)
    (6): Linear(in_features=5, out_features=5, bias=True)
  )
  (decoder): Linear(in_features=5, out_features=5, bias=True)
)
9520


TypeError: linear(): argument 'input' (position 1) must be Tensor, not numpy.ndarray

In [ ]:
# plot the losses vs epoch

A helper function to retrieve the individual event-wise losses from the sample (`getLosses`) 

In [ ]:
net.eval()
def getLosses(sample):
    losses=[]
    for i in range(len(sample)):
        pred = net(sample[i])
        loss = loss_fn(pred, sample[i])
        losses.append(loss.data.item())
    return losses

In [ ]:
# get the training dataset losses and plot them

In [ ]:
# get the test dataset losses and plot them

In [ ]:
# make a single plot overlaying the training and test dataset losses, 
# with each normalized so they both have unit area.